## Diversity Profile

Goal: decomposition of diversity dimensions to allow a comprehensive evaluation

### Load Imports

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import random
import os

from tqdm import tqdm
import time

from importlib import reload

In [ ]:
import src.coverage as cov
import src.frequency_profiling as occ

### 0 Create multiple toy datasets for different diversity scenarios

These toy datasets can be used for test scenarios

#### 0.1 Manual toy datasets for controlled outcomes

In [ ]:
# test case: dataframe with all static columns each with 1 value
df_static = pd.DataFrame({'model': ['_a']*1000,
                     'manufacturer': ['_b']*1000,
                        'version': ['_c']*1000,
                            'country': ['_d']*1000,
                                'region': ['_e']*1000,
                                    'wholesaler': ['_f']*1000,})

df_static.head()

In [ ]:
df_static.to_csv('data/df_static.csv', index=False)

There is only one combination / one pattern in the data. But this pattern is covered. And all of its children patterns are also covered. We have no uncovered patterns (or mups). Hence coverage is 100%.


However, the diversity must still be low because it is only one pattern in the data. -> low variance, high coverage

In [ ]:
# test case: 10 categorical columns with random data

# Define the categories for each attribute
categories = {
    'Attribute1': ['A', 'B', 'C', 'D'],
    'Attribute2': ['X', 'Y', 'Z'],
    'Attribute3': ['M', 'N', 'O', 'P', 'Q'],
    'Attribute4': ['Alpha', 'Beta'],
    'Attribute5': ['Red', 'Green', 'Blue', 'Yellow'],
    'Attribute6': ['One', 'Two', 'Three', 'Four', 'Five', 'Six'],
    'Attribute7': ['Apple', 'Banana', 'Orange'],
    'Attribute8': ['Up', 'Down'],
    'Attribute9': ['First', 'Second', 'Third', 'Fourth'],
    'Attribute10': ['Dog', 'Cat', 'Bird', 'Fish', 'Rabbit']
}

# Create an empty DataFrame
df_skewed = pd.DataFrame()

# Generate random data for each attribute
for attribute, category_list in categories.items():
    if attribute in ['Attribute1', 'Attribute2']:
        # Generate uniform distribution
        weights = np.ones(len(category_list))
        weights = weights / np.sum(weights)
        print(attribute, weights)
        random_categories = np.random.choice(category_list, size=1000, replace=True, p=weights)
    else:
        # Generate highly skewed distribution
        weights = np.random.rand(len(category_list))
        weights = weights / np.sum(weights)   # randomly adjust weights to control skewness
        print(attribute, weights)
        random_categories = np.random.choice(category_list, size=1000, replace=True, p=weights)
    
    df_skewed[attribute] = random_categories

In [ ]:
df_skewed.to_csv('data/df_skewed.csv', index=False)

In [ ]:
df_uniform = pd.DataFrame()

categories_diverse = {
    'Attribute1': ['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon', 'Zeta', 'Eta', 'Theta', 'Iota', 'Kappa'],
    'Attribute2': ['Red', 'Green', 'Blue', 'Yellow'],
    'Attribute3': ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine'], # 'Ten', 'Eleven'],
    'Attribute4': ['Apple', 'Banana', 'Orange'],
    'Attribute5': ['Up', 'Down'],
}

for attribute, category_list in categories_diverse.items():
    weights = np.ones(len(category_list))
    weights = weights / np.sum(weights)
    print(attribute, weights)
    random_categories = np.random.choice(category_list, size=1000, replace=True, p=weights)
    df_uniform[attribute] = random_categories

In [ ]:
df_uniform.head()

In [ ]:
df_uniform.to_csv('data/df_uniform.csv', index=False)

In [ ]:
# Toy dataset where every combination of attributes from categories_diverse is covered
# Generate all attribute combinations
attribute_combinations = list(itertools.product(*categories_diverse.values()))
# Shuffle the combinations and add random duplicates
random.shuffle(attribute_combinations)
attribute_combinations = attribute_combinations + random.choices(attribute_combinations, k=1000)

# Create DataFrame
df_covered = pd.DataFrame(attribute_combinations, columns=categories_diverse.keys())    

In [ ]:
df_covered.to_csv('data/df_covered.csv', index=False)

In [ ]:
df_covered.head()

#### 0.2 Create toy dataset with synthesized library for scalability tests

In [ ]:
### large dataframe with 1,500,000 [large] // 150,000 rows [medium] and 13 columns
if os.path.isfile('data/adult_synth_medium.csv'):
    print("File exists")
    df_synth = pd.read_csv('data/adult_synth_large.csv')
else:
    print("File does not exist -> Generate new synthetic data")
    df = df_adult.sample(n=1500, random_state=1)
    df_meta = synthesized.MetaExtractor.extract(df)
    synthesizer = synthesized.HighDimSynthesizer(df_meta)  
    synthesizer.learn(df)
    df_synth = synthesizer.synthesize(num_rows=150000)
    df_synth.to_csv('data/adult_synth_medium.csv', index=False)

In [ ]:
df_synth.value_counts()

In [ ]:
df_synth = df_synth.drop(columns=['education']) # currently to avoid cardinality issues

In [ ]:
### random dataframe with 14 columns and 1,000 rows
df_synth_short = df_synth.sample(n=1000, random_state=1).reset_index(drop=True)

#### 0.3 Choose one dataset to continue evaluation with

In [ ]:
df = pd.read_csv("data/adult_synth_medium.csv")
df.drop(columns=['native-country'], inplace=True)
df = df.iloc[:,:5]
df.columns = [s.replace('-', '_') for s in df.columns]

In [ ]:
# df = df_skewed.copy()
# df = df.drop(columns)
# df = df_synth.iloc[:,:5].copy()
# df = df_compas.copy()
df = df_uniform.copy()
df.columns = [s.replace('-', '_') for s in df.columns]

In [ ]:
df = df.replace({'?': '--'})

In [ ]:
df.head()

,hours_per_week,age,capital_gain,capital_loss,workclass
0,40 to 60,Less than 25,No,No,Private
1,40 to 60,25 to 45,No,No,Private
2,40 to 60,25 to 45,No,No,Private
3,More than 60,Greater than 45,No,Yes,Private
4,40 to 60,Greater than 45,No,No,--


In [ ]:
for col in tqdm(range(len(df.columns))):
    labels, uniques = pd.factorize(df[df.columns[col]])
    labels = [str(col) + ":" + str(l) for l in labels]
    df[df.columns[col]] = labels
del labels, uniques

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:01<00:00,  4.46it/s]


### 1 Add real world examples as reference

#### 1.0 Replication of data from ProPublica article (Angwin et al. 2016) 

In [ ]:
df_compas_raw = pd.read_csv('data/compas-scores-two-years.csv')
df_compas = (df_compas_raw[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 
             'sex', 'priors_count', 'decile_score', 'two_year_recid']]
             .loc[(df_compas_raw['days_b_screening_arrest'] <= 30) & (df_compas_raw['days_b_screening_arrest'] >= -30), :]
             .loc[df_compas_raw['is_recid'] != -1, :]
             .loc[df_compas_raw['c_charge_degree'] != 'O', :]
             .loc[df_compas_raw['score_text'] != 'N/A', :]
             )
print('Number of rows: {}'.format(len(df_compas.index)))


In [ ]:
# turn compas data into categorical data
df_compas['age'] = df_compas['age'].apply(lambda x: 'Less than 25' if x < 25 else '25 to 45' if x < 45 else 'Greater than 45')
df_compas['age'] = df_compas['age'].astype('category')

df_compas['two_year_recid'] = df_compas['two_year_recid'].replace({0: 'No', 1: 'Yes'})

df_compas['decile_score'] = df_compas['decile_score'].apply(lambda x: 'Low' if x <= 4 else 'Medium' if x <= 7 else 'High')

df_compas['priors_count'] = df_compas['priors_count'].apply(lambda x: str(x) if x <= 5 else 'More than 5')

In [ ]:
df_compas.to_csv('data/df_compas.csv', index=False)

In [ ]:
df_compas.head()

#### 1.1 ACS Income Dataset

Use ACSIncome and categorize as closely to Adult Dataset as possible to allow for diversity comparison and evaluation

In [37]:
from folktables import ACSDataSource, ACSIncome, adult_filter
import folktables

ACSIncome_100000 = folktables.BasicProblem(
    features=[
        'AGEP',
        'COW',
        'SCHL',
        'MAR',
        'OCCP',
        'POBP',
        'RELP',
        'WKHP',
        'SEX',
        'RAC1P',
    ],
    target='PINCP',
    # target_transform=lambda x: x > 100000,
    group='RAC1P',
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

In [89]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
ca_data = data_source.get_data(download=True)

In [90]:
ca_features, ca_labels, _ = ACSIncome_100000.df_to_pandas(ca_data)

In [91]:
ca_features.join(ca_labels).to_csv('data/df_ACSIncome_raw.csv', index=False)

In [93]:
acs_income = ca_features.join(ca_labels)

In [71]:
acs_income = pd.read_csv('data/df_ACSIncome_CA_raw.csv')

In [94]:
acs_income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP
0,18.0,1.0,18.0,5.0,4720.0,13.0,17.0,21.0,2.0,2.0,1600.0
1,53.0,5.0,17.0,5.0,3605.0,18.0,16.0,40.0,1.0,1.0,10000.0
2,41.0,1.0,16.0,5.0,7330.0,1.0,17.0,40.0,1.0,1.0,24000.0
3,18.0,6.0,18.0,5.0,2722.0,1.0,17.0,2.0,2.0,1.0,180.0
4,21.0,5.0,19.0,5.0,3870.0,12.0,17.0,50.0,1.0,1.0,29000.0


In [95]:
acs_income['age'] = acs_income['AGEP'].apply(lambda x: 'Less than 25' if x < 25 else '25 to 45' if (x >= 25 and x <= 45) else 'Greater than 45')
acs_income['age'] = acs_income['age'].astype('category')

In [ ]:
df_adult['workclass'].unique()

In [96]:
COW = {
    1: 'Private',
    2: 'Private',
    3: 'Local-gov',
    4: 'State-gov',
    5: 'Federal-gov',
    6: 'Self-emp-not-inc',
    7: 'Self-emp-inc',
    8: 'Without-pay',
    9: 'Never-worked'
}

acs_income['workclass'] = acs_income['COW'].replace(COW)
acs_income['workclass'] = acs_income['workclass'].astype('category')

In [97]:
acs_income.head()

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,PINCP,age,workclass
0,18.0,1.0,18.0,5.0,4720.0,13.0,17.0,21.0,2.0,2.0,1600.0,Less than 25,Private
1,53.0,5.0,17.0,5.0,3605.0,18.0,16.0,40.0,1.0,1.0,10000.0,Greater than 45,Federal-gov
2,41.0,1.0,16.0,5.0,7330.0,1.0,17.0,40.0,1.0,1.0,24000.0,25 to 45,Private
3,18.0,6.0,18.0,5.0,2722.0,1.0,17.0,2.0,2.0,1.0,180.0,Less than 25,Self-emp-not-inc
4,21.0,5.0,19.0,5.0,3870.0,12.0,17.0,50.0,1.0,1.0,29000.0,Less than 25,Federal-gov


In [ ]:
df_adult['education'].unique()

In [98]:
SCHL = {
    1: '?',
    2: 'Preschool',
    3: 'Preschool',
    4: '1st-4th',
    5: '1st-4th',
    6: '1st-4th',
    7: '1st-4th',
    8: '5th-6th',
    9: '5th-6th',
    10: '7th-8th',
    11: '7th-8th',
    12: '9th',
    13: '10th',
    14: '11th',
    15: '12th',
    16: 'HS-grad',
    17: 'HS-grad',
    18: 'Some-college',
    19: 'Some-college',
    20: 'Assoc-voc',
    21: 'Bachelors',
    22: 'Masters',
    23: 'Prof-school',
    24: 'Doctorate',
}

acs_income['education'] = acs_income['SCHL'].replace(SCHL)
acs_income['education'] = acs_income['education'].astype('category')

In [93]:
df_adult['marital-status'].unique()

array(['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated',
       'Widowed', 'Married-spouse-absent', 'Married-AF-spouse'],
      dtype=object)

In [99]:
MAR = {
    1: 'Married-civ-spouse',
    2: 'Widowed',
    3: 'Divorced',
    4: 'Separated',
    5: 'Never-married'
}

acs_income['marital-status'] = acs_income['MAR'].replace(MAR)
acs_income['marital-status'] = acs_income['marital-status'].astype('category')

In [96]:
df_adult['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [100]:
SEX = {
    1: "Male",
    2: "Female"
}

acs_income['gender'] = acs_income['SEX'].replace(SEX)
acs_income['gender'] = acs_income['gender'].astype('category')

In [98]:
df_adult['native-country'].unique()

array(['United-States', 'Cambodia', '?', 'England', 'Puerto-Rico',
       'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India',
       'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras',
       'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico',
       'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos',
       'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala',
       'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador',
       'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands'],
      dtype=object)

In [101]:
POBP = {
100: "Albania",
102: "Austria",
103: "Belgium",
104: "Bulgaria",
105: "Czechoslovakia",
106: "Denmark",
108: "Finland",
109: "France",
110: "Germany",
116: "Greece",
117: "Hungary",
118: "Iceland",
119: "Ireland",
120: "Italy",
126: "Netherlands",
127: "Norway",
128: "Poland",
129: "Portugal",
130: "Azores Islands",
132: "Romania",
134: "Spain",
136: "Sweden",
137: "Switzerland",
138: "United Kingdom, Not Specified",
139: "England",
140: "Scotland",
142: "Northern Ireland (2017 or later)",
147: "Yugoslavia",
148: "Czech Republic",
149: "Slovakia",
150: "Bosnia and Herzegovina",
151: "Croatia",
152: "Macedonia",
154: "Serbia",
156: "Latvia",
157: "Lithuania",
158: "Armenia",
159: "Azerbaijan",
160: "Belarus",
161: "Georgia",
162: "Moldova",
163: "Russia",
164: "Ukraine",
165: "USSR",
166: "Europe (2017 or later)",
167: "Kosovo (2017 or later)",
168: "Montenegro",
169: "Other Europe, Not Specified",
200: "Afghanistan",
202: "Bangladesh",
203: "Bhutan",
205: "Myanmar",
206: "Cambodia",
207: "China",
208: "Cyprus (2016 or earlier)",
209: "Hong Kong",
210: "India",
211: "Indonesia",
212: "Iran",
213: "Iraq",
214: "Israel",
215: "Japan",
216: "Jordan",
217: "Korea",
218: "Kazakhstan",
219: "Kyrgyzstan (2017 or later)",
222: "Kuwait",
223: "Laos",
224: "Lebanon",
226: "Malaysia",
228: "Mongolia (2017 or later)",
229: "Nepal",
231: "Pakistan",
233: "Philippines",
235: "Saudi Arabia",
236: "Singapore",
238: "Sri Lanka",
239: "Syria",
240: "Taiwan",
242: "Thailand",
243: "Turkey",
245: "United Arab Emirates",
246: "Uzbekistan",
247: "Vietnam",
248: "Yemen",
249: "Asia",
253: "South Central Asia, Not Specified",
254: "Other Asia, Not Specified",
300: "Bermuda",
301: "Canada",
303: "Mexico",
310: "Belize",
311: "Costa Rica",
312: "El Salvador",
313: "Guatemala",
314: "Honduras",
315: "Nicaragua",
316: "Panama",
321: "Antigua and Barbuda",
323: "Bahamas",
324: "Barbados",
327: "Cuba",
328: "Dominica",
329: "Dominican Republic",
330: "Grenada",
332: "Haiti",
333: "Jamaica",
338: "St. Kitts-Nevis (2017 or later)",
339: "St. Lucia",
340: "St. Vincent and the Grenadines",
341: "Trinidad and Tobago",
343: "West Indies",
344: "Caribbean, Not Specified",
360: "Argentina",
361: "Bolivia",
362: "Brazil",
363: "Chile",
364: "Colombia",
365: "Ecuador",
368: "Guyana",
369: "Paraguay",
370: "Peru",
372: "Uruguay",
373: "Venezuela",
374: "South America",
399: "Americas, Not Specified",
400: "Algeria",
407: "Cameroon",
408: "Cabo Verde",
412: "Congo",
414: "Egypt",
416: "Ethiopia",
417: "Eritrea",
420: "Gambia",
421: "Ghana",
423: "Guinea",
425: "Ivory Coast (2017 or later)",
427: "Kenya",
429: "Liberia",
430: "Libya",
436: "Morocco",
440: "Nigeria",
442: "Rwanda (2017 or later)",
444: "Senegal",
447: "Sierra Leone",
448: "Somalia",
449: "South Africa",
451: "Sudan",
453: "Tanzania",
454: "Togo",
456: "Tunisia (2017 or later)",
457: "Uganda",
459: "Democratic Republic of Congo (Zaire)",
460: "Zambia",
461: "Zimbabwe",
462: "Africa",
463: "South Sudan (2017 or later)",
464: "Northern Africa, Not Specified",
467: "Western Africa, Not Specified",
468: "Other Africa, Not Specified",
469: "Eastern Africa, Not Specified",
501: "Australia",
508: "Fiji",
511: "Marshall Islands",
512: "Micronesia",
515: "New Zealand",
523: "Tonga",
527: "Samoa",
554: "Other US Island Areas, Oceania, Not Specified, or at Sea",
}

In [102]:
acs_income['native-country'] = acs_income['POBP'].apply(lambda x: 'United-States' if x < 100 else x)
acs_income['native-country'] = acs_income['native-country'].replace(POBP)

In [103]:
#replace all except the 49 highest frequency countries with 'Other'
for country in acs_income['native-country'].value_counts().index[49:]:
    acs_income['native-country'] = acs_income['native-country'].replace(country, 'Other')

In [107]:
df_adult['race'].unique()

array(['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other',
       'Black'], dtype=object)

In [104]:
RACE = {
    1: 'White',
    2: 'Black',
    3: 'American Indian',
    4: 'Alaska Native',
    5: 'Amer-Indian-Native-Specified',
    6: 'Asian',
    7: 'Asian-Pac-Islander',
    8: 'Other',
    9: 'Two or more races',
}

In [105]:
acs_income['race'] = acs_income['RAC1P'].replace(RACE)

In [106]:
acs_income['race'] = acs_income['race'].astype('category')

In [65]:
acs_income.columns

Index(['AGEP', 'COW', 'SCHL', 'MAR', 'OCCP', 'POBP', 'RELP', 'WKHP', 'SEX',
       'RAC1P', 'PINCP', 'age', 'workclass', 'education', 'marital-status',
       'gender', 'native-country', 'race'],
      dtype='object')

In [140]:
#bucketize PINCP in 10,000 dollar increments and convert to category (e.g. "0-10,000", "10,000-20,000", etc.)
acs_income["income"] = pd.cut(acs_income["PINCP"], bins=np.arange(0, 500000, 10000), labels=[f"${i}-{i+10}k" for i in range(0, 490, 10)])
#fill in missing values with above 500k category
acs_income["income"] = acs_income["income"].cat.add_categories(f"$490k+")
acs_income["income"] = acs_income["income"].fillna("$490k+")

In [141]:
acs_income['income'] = acs_income['income'].astype('category')

In [144]:
acs_income.nunique()

AGEP                 80
COW                   8
SCHL                 24
MAR                   5
OCCP                529
POBP                224
RELP                 18
WKHP                 99
SEX                   2
RAC1P                 9
PINCP             18107
age                   3
workclass             7
education            16
marital-status        5
gender                2
native-country       50
race                  9
income               50
dtype: int64

In [38]:
# acs_income['income'] = acs_income['PINCP'].apply(lambda x: '<=50K' if x == False else '>50K')

In [115]:
df_adult['hours-per-week'].unique()

['Less than 40', '40 to 60', 'More than 60']
Categories (3, object): ['40 to 60', 'Less than 40', 'More than 60']

In [145]:
acs_income['hours-per-week'] = acs_income['WKHP'].apply(lambda x: 'Less than 40' if x < 40 else '40 to 60' if (x >= 40 and x<= 60) else 'More than 60')

In [ ]:
set(df_adult.columns)-set(acs_income.columns)

In [88]:
len(acs_income)

195665

In [87]:
acs_income[['age', 'workclass', 'education', 'marital-status',
       'gender', 'native-country', 'race', 'income', 'hours-per-week']].to_csv('data/df_ACSIncome.csv', index=False)

In [10]:
acs_income_mexico = acs_income.loc[acs_income['native-country'] == 'Mexico', :]

In [12]:
acs_income_mexico.to_csv('data/df_ACSIncome_Mexico.csv', index=False)

In [11]:
len(acs_income_mexico)

57275

#### 1.2 Add Reference Dataset for Adult Dataset (UCI Adult dataset)
Folktables Adult Dataset should be an updated version of UCI Adult dataset.

Therefore uci_adult can be reference for ACSIncome (and have similar diversity profile)

In [ ]:
uci_adult_reference_link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

In [ ]:
features = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
        "occupation", "relationship", "race", "gender", "capital-gain", "capital-loss",
        "hours-per-week", "native-country", "income"] 

In [ ]:
uci_adult = pd.read_csv(uci_adult_reference_link, names=features, sep=r'\s*,\s*', 
                             engine='python', na_values="?")

In [ ]:
uci_adult = uci_adult.drop(['fnlwgt'], axis=1)

In [ ]:
uci_adult.fillna("?", inplace=True)

In [ ]:
len(uci_adult)

32561

In [ ]:
uci_adult.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25 to 45,State-gov,Bachelors,More than 12,Never-married,Adm-clerical,Not-in-family,White,Male,Yes,No,40 to 60,United-States,<=50K
1,Greater than 45,Self-emp-not-inc,Bachelors,More than 12,Married-civ-spouse,Exec-managerial,Husband,White,Male,No,No,Less than 40,United-States,<=50K
2,25 to 45,Private,HS-grad,8 to 12,Divorced,Handlers-cleaners,Not-in-family,White,Male,No,No,40 to 60,United-States,<=50K
3,Greater than 45,Private,11th,Less than 8,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,No,No,40 to 60,United-States,<=50K
4,25 to 45,Private,Bachelors,More than 12,Married-civ-spouse,Prof-specialty,Wife,Black,Female,No,No,40 to 60,Cuba,<=50K


In [ ]:
uci_adult['hours-per-week'] = uci_adult['hours-per-week'].astype('int')
uci_adult['age'] = uci_adult['age'].astype('int')
uci_adult['education-num'] = uci_adult['education-num'].astype('int')
uci_adult['capital-gain'] = uci_adult['capital-gain'].astype('int')
uci_adult['capital-loss'] = uci_adult['capital-loss'].astype('int')

In [ ]:
uci_adult['age'] = uci_adult['age'].apply(lambda x: 'Less than 25' if x < 25 else '25 to 45' if x < 45 else 'Greater than 45')
uci_adult['age'] = uci_adult['age'].astype('category')

uci_adult['hours-per-week'] = uci_adult['hours-per-week'].apply(lambda x: 'Less than 40' if x < 40 else '40 to 60' if x < 60 else 'More than 60')
uci_adult['hours-per-week'] = uci_adult['hours-per-week'].astype('category')

uci_adult['education-num'] = uci_adult['education-num'].apply(lambda x: 'Less than 8' if x < 8 else '8 to 12' if x < 12 else 'More than 12')
uci_adult['education-num'] = uci_adult['education-num'].astype('category')

uci_adult['capital-gain'] = uci_adult['capital-gain'].apply(lambda x: 'No' if x == 0 else 'Yes')
uci_adult['capital-gain'] = uci_adult['capital-gain'].astype('category')

uci_adult['capital-loss'] = uci_adult['capital-loss'].apply(lambda x: 'No' if x == 0 else 'Yes')
uci_adult['capital-loss'] = uci_adult['capital-loss'].astype('category')

In [ ]:
for col in uci_adult.columns:
    print(f"{col}: {set(uci_adult[col].unique())-set(df_adult[col].unique())}")

age: set()
workclass: set()
education: set()
education-num: set()
marital-status: set()
occupation: set()
relationship: set()
race: set()
gender: set()
capital-gain: set()
capital-loss: set()
hours-per-week: set()
native-country: set()
income: set()


In [ ]:
uci_adult.to_csv("data/df_uci_adult.csv", index=False)

##### Reference Projection for Adult Data

In [5]:
# df_adult with only gender=male and race=white
df_adult = pd.read_csv("data/df_uci_adult.csv")
df_adult_mw = df_adult[(df_adult['gender']=='Male') & (df_adult['race']=='White')]

In [8]:
df_adult_mw.to_csv('data/df_uci_adult_mw.csv', index=False)

In [6]:
all_occurences, keys = occ.get_occurence_count(df_adult_mw, max_level=4)


Using row-based traversal
using row-based traversal


100%|██████████| 7700/7700 [00:36<00:00, 208.16it/s]


In [9]:
all_occurences["('age:Less than 25',)"]

0

In [7]:
cov.get_occurences_of_parent_patterns(('age:Less than 25', 'education:Doctorate'), all_occurences)

{('age:Less than 25',): 0, ('education:Doctorate',): 0}

#### 1.3 Add UK Road Safety Dataset

Download UKRoadSafety data from https://www.kaggle.com/datasets/tsiaras/uk-road-safety-accidents-and-vehicles/data. We use the Accident Information data.

In [ ]:
df_accident_info = pd.read_csv("data/UK_RoadSafety_Accident_Information.csv")

In [13]:
#get all object columns
df_accident_info_col = df_accident_info.select_dtypes(include=['object'])
df_accident_info_col.nunique()

Accident_Index                2047256
1st_Road_Class                      6
2nd_Road_Class                      6
Accident_Severity                   3
Carriageway_Hazards                 7
Date                             4748
Day_of_Week                         7
Junction_Control                    6
Junction_Detail                    10
Light_Conditions                    6
Local_Authority_(District)        416
Local_Authority_(Highway)         207
LSOA_of_Accident_Location       35564
Police_Force                       51
Road_Surface_Conditions             6
Road_Type                           7
Special_Conditions_at_Site          9
Time                             1439
Urban_or_Rural_Area                 3
Weather_Conditions                 10
InScotland                          2
dtype: int64

In [16]:
df_accident_info = df_accident_info[["Light_Conditions", "Weather_Conditions", "Road_Type", "Day_of_Week", "Speed_limit", "Urban_or_Rural_Area", "Accident_Severity", "Road_Surface_Conditions", "Year"]]

In [153]:
# get dtype of all columns
df_accident_info.dtypes

Light_Conditions            object
Weather_Conditions          object
Road_Type                   object
Day_of_Week                 object
Speed_limit                float64
Urban_or_Rural_Area         object
Accident_Severity           object
Road_Surface_Conditions     object
Year                         int64
dtype: object

In [17]:
# get cardinality of every column
df_accident_info.nunique()

Light_Conditions            6
Weather_Conditions         10
Road_Type                   7
Day_of_Week                 7
Speed_limit                 9
Urban_or_Rural_Area         3
Accident_Severity           3
Road_Surface_Conditions     6
Year                       13
dtype: int64

In [19]:
len(df_accident_info.drop_duplicates())

119991

In [20]:
df_accident_info.columns

Index(['Light_Conditions', 'Weather_Conditions', 'Road_Type', 'Day_of_Week',
       'Speed_limit', 'Urban_or_Rural_Area', 'Accident_Severity',
       'Road_Surface_Conditions', 'Year'],
      dtype='object')

In [155]:
df_accident_info.to_csv("data/df_uk_road_accident.csv", index=False)

In [156]:
df_accident_info_enc = occ.encode_data_pure(df_accident_info)
df_accident_info_enc.to_csv("data/df_uk_road_accident_enc.csv", index=False)

100%|██████████| 9/9 [00:11<00:00,  1.33s/it]


#### 1.4 Add BlueNile dataset

Download the BlueNile dataset from the CoverageJava repo of the baseline DeepDiver algorithm: https://github.com/UIC-InDeXLab/MithraLabel/tree/master/CoverageJava/data

In [3]:
bn_raw = pd.read_csv("CoverageJava/data/BN-2-21-2018-n116300.csv")

In [4]:
bn_raw.columns

Index(['id', 'Carat', 'Clarity', 'Color', 'Cut', 'Depth', 'Fluorescence', 'ID',
       'LengthWidthRatio', 'Polish', 'Price', 'Shape', 'Symmetry', 'Table',
       'Carat_norm', 'Clarity_norm', 'Color_norm', 'Cut_norm', 'Depth_norm',
       'LengthWidthRatio_norm', 'Price_inv_norm', 'Table_norm', 'Rand#'],
      dtype='object')

In [5]:
bn = bn_raw[["Shape", "Color", "Cut", "Clarity", "Polish", "Symmetry", "Fluorescence"]]

In [7]:
bn.to_csv("data/df_diamonds.csv", index=False)

Use dataset from CoverageJava paper

In [11]:
import pandas as pd

In [12]:
bn_raw_num = pd.read_csv("data/bluenile_raw.csv")
bn_num = bn_raw_num[bn_raw_num.columns[1:]]
bn_num.columns = ["Shape", "Color", "Cut", "Clarity", "Polish", "Symmetry", "Fluorescence"]
bn_num.to_csv("data/df_num_diamonds_raw.csv", index=False)

In [14]:
import src.occurence_estimation as occ
bn_num_cat, mapping = occ.factorize_data(bn_num)
bn_num_cat.to_csv("data/df_num_diamonds_enc_num.csv", index=False)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:01<00:00,  4.60it/s]


#### 1.5 Add IMDb dataset

source: https://developer.imdb.com/non-commercial-datasets/

In [84]:
# reading given tsv file
imdb_table=pd.read_table("data/imdb_basics_data.tsv", index_col="tconst", na_values="\\N")

/var/folders/7m/gfw5j53x1hx29cv80bbnf50m0000gp/T/ipykernel_98882/2560924856.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_table=pd.read_table("data/imdb_basics_data.tsv", index_col="tconst", na_values="\\N")


In [86]:
imdb_table = imdb_table[["titleType", "isAdult", "startYear", "runtimeMinutes", "genres"]]

In [103]:
imdb_table.nunique()

titleType         10
isAdult            2
startYear         18
runtimeMinutes     7
genres            28
dtype: int64

In [88]:
# filter out rows with unaligned data
imdb_table = imdb_table[imdb_table.isAdult < 2]

In [89]:
# bucketize startYear in 10 year buckets (e.g. 1990-1999)
imdb_table["startYear"] = imdb_table["startYear"].apply(lambda x: str(x)[:3] + "0s")

In [97]:
pd.DataFrame(imdb_table["runtimeMinutes"].value_counts()).to_clipboard()

In [96]:
#categorize runtimeMinutes into 30 min buckets
imdb_table["runtimeMinutes"] = imdb_table["runtimeMinutes"].astype(float).apply(lambda x: x if pd.isna(x) else "<30min" if x < 30 else "30-60min" if x < 60 else "60-90min" if x < 90 else "90-120min" if x < 120 else ">120min" if x < 180 else ">180min")

In [98]:
#replace NaN values in runtimeMinutes with "unknown"
imdb_table["runtimeMinutes"] = imdb_table["runtimeMinutes"].fillna("unknown")

In [100]:
#remove all rows where genres is not string
imdb_table = imdb_table[imdb_table.genres.apply(lambda x: isinstance(x, str))]

In [101]:
# reduce number of genres by only using first genre
imdb_table["genres"] = imdb_table["genres"].apply(lambda x: x.split(",")[0])

In [105]:
#drop index column
imdb_table = imdb_table.reset_index(drop=True)

In [106]:
imdb_table.to_csv("data/imdb_data.csv")

#### 1.6 add IMDb Top 7000 dataset

source: https://www.kaggle.com/datasets/mazenramadan/imdb-most-popular-films-and-series

In [4]:
imdb_top = pd.read_csv("data/imdb.csv")

In [19]:
imdb_top.nunique()

Name           4820
Date             11
Rate             10
Votes          4802
Genre            23
Duration          7
Type              2
Certificate      23
Episodes        284
Nudity            5
Violence          5
Profanity         5
Alcohol           5
Frightening       5
dtype: int64

In [12]:
#Reduce number of genres by only using first genre
imdb_top["Genre"] = imdb_top["Genre"].apply(lambda x: x.split(",")[0])

In [9]:
#Categorize Rate to buckets of 0-10 in steps of 1 as "0-1" etc.
imdb_top["Rate"] = imdb_top["Rate"].apply(lambda x: str(int(float(x))) + "-" + str(int(float(x))+1) if x != "No Rate" else x)


In [15]:
#Reduce duration to buckets of 30 min
imdb_top["Duration"] = imdb_top["Duration"].apply(lambda x: x if x=="None" else "<30min" if float(x) < 30 else "30-60min" if float(x) < 60 else "60-90min" if float(x) < 90 else "90-120min" if float(x) < 120 else ">120min" if float(x) < 180 else ">180min")

In [18]:
#Reduce number of years to buckets of 10 years
imdb_top["Date"] = imdb_top["Date"].apply(lambda x: str(x)[:3] + "0s")

In [ ]:
imdb_top.to_csv("data/imdb.csv", index=False)

#### 1.7 add Covertype Dataset

Test with ecology dataset. Source is https://networkrepository.com/covtype.php

In [29]:
covertype = pd.read_csv("data/covtype.data")

In [30]:
covertype.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [31]:
#turn binary columns Soil_Type1 to Soil_Type40 into one categorical column "Soil Type" with values 1-40
covertype["SoilType"] = pd.Series(covertype[covertype.columns[14:54]].idxmax(axis=1)).apply(lambda x: int(x.split("e")[1]))

In [32]:
#turn binary columns Wilderness_Area1 to Wilderness_Area4 into one categorical column "Wilderness Area" with values 1-4
covertype["WildernessArea"] = pd.Series(covertype[covertype.columns[10:14]].idxmax(axis=1)).apply(lambda x: int(x.split("a")[1]))

In [33]:
#drop Soil_Type1 to Soil_Type40 and Wilderness_Area1 to Wilderness_Area4
covertype = covertype.drop(columns=covertype.columns[10:54])

In [34]:
#bucketize Elevation into 500m buckets (e.g. 2000-2500)
covertype["Elevation"] = covertype["Elevation"].apply(lambda x: str(int(x/500)*500) + "-" + str(int(x/500)*500+500))

In [35]:
covertype.Elevation.value_counts()

3000-3500    284467
2500-3000    252947
2000-2500     39800
3500-4000      2540
1500-2000      1258
Name: Elevation, dtype: int64

In [36]:
covertype.to_csv("data/covertype.csv", index=False)

### 2 Save all selected Datasets in encoded versions
- ACSIncome
- ACSIncome, filtered for CA
- UKRoadSafety
- BlueNile

In [3]:
acs_income_ca = pd.read_csv('data/df_ACSIncome_CA.csv')

In [ ]:
acs_income_ca_enc = occ.encode_data_pure(acs_income_ca)
acs_income_ca_enc.to_csv('data/df_ACSIncome_CA_enc.csv', index=False)

In [5]:
acs_income = pd.read_csv('data/df_ACSIncome.csv')

In [6]:
acs_income_enc = occ.encode_data_pure(acs_income)
acs_income_enc.to_csv('data/df_ACSIncome_enc.csv', index=False)

100%|██████████| 9/9 [00:08<00:00,  1.12it/s]


In [7]:
uk_roadsafety = pd.read_csv('data/df_uk_road_accident.csv')

In [8]:
uk_roadsafety_enc = occ.encode_data_pure(uk_roadsafety)
uk_roadsafety_enc.to_csv('data/df_uk_road_accident_enc.csv', index=False)

100%|██████████| 8/8 [00:08<00:00,  1.12s/it]


In [91]:
acs_income_ca_enc_select = acs_income_ca_enc[["education", "gender", "income", "race"]]

In [95]:
freq_count, keys = occ.calc_occurences_countmin_rowbased_traversal(acs_income_ca_enc_select)

using row-based traversal


  0%|          | 0/2736 [00:00<?, ?it/s]

100%|██████████| 2736/2736 [00:00<00:00, 9860.28it/s]


In [98]:
frequencies_general = occ.get_general_occurences(list(acs_income_ca_enc_select.columns), keys, freq_count)

100%|██████████| 4/4 [00:00<00:00, 90.11it/s]
